In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [53]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ',
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = str.strip('\n')
    str = str.strip(' ')

    return str

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./STTRResult/03_STTR_phase1.csv', index=False, encoding="utf-8")
    
def ReadFiles(Directory_path):  # Read all the html files
    path = Directory_path  # The directory
    file_names = os.listdir(path)  # 得到文件夹下的所有文件名称
    files_position = []
    for file_name in file_names:
        position = path + "/" + file_name
        # print(position)
        files_position.append(position)

    print("Total number of HTML files: ", len(files_position))
    return files_position


def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

In [52]:
def SingleHTMLProcess(path):
    dic = {"Proposal Number": "", "Phase 1 Contract #": "", "Subtopic Title": "", "Proposal Title": "",
           "Small Business Concern_Firm": "", "Small Business Concern_Address": "", "Small Business Concern_Phone": "",
           "Principal Investigator_Name": "", "Principal Investigator_E-mail": "", "Principal Investigator_Address": "", "Is Principal Investigator US Citizen or Permanent Resident":"",
           "Principal Investigator_Phone": "", "Principal Investigator_Company": "",
           "Business Official_Name": "", "Business Official_E-mail": "", "Business Official_Address": "",
           "Business Official_Phone": "",
           "TRL_Begin": "", "TRL_End": "", "Technical Abstract": "", "Potential NASA applications": "",
           "Potential non-NASA applications": "","Technology Taxonomy Mapping":"", 'Research Topic':"", 'Research Institution Name':"", 'Research Institution Address':"",
          "Research Institution Phone":""}
    htmlfile = open(path, 'r', encoding='utf-8')
    html=htmlfile  
    bs = BeautifulSoup(html, "lxml")  # 缩进格式
    print(path)
    dic['Technical Abstract'] = filterHTMLstr(getTechAbstract(bs))
    dic['Potential NASA applications'] = filterHTMLstr(getNonNASAApplication(bs))
    dic['Potential non-NASA applications'] = filterHTMLstr(getNonNASAApplication(bs))
    dic['Proposal Number'] = filterHTMLstr(getProposalNumber(bs))
    dic['Subtopic Title'] = filterHTMLstr(getResearchSubTopicTitle(bs))
    dic['Proposal Title'] = filterHTMLstr(getProjectTitle(bs))
    smbAndRIInfo = getSMBAndRIInfo(bs)
    dic['Small Business Concern_Firm'] = filterHTMLstr(smbAndRIInfo[0])
    dic['Small Business Concern_Address'] = filterHTMLstr(smbAndRIInfo[1])
    dic['Small Business Concern_Phone'] = filterHTMLstr(smbAndRIInfo[2])
    dic['Research Institution Name'] = filterHTMLstr(smbAndRIInfo[3])
    dic['Research Institution Address'] = filterHTMLstr(smbAndRIInfo[4])
    dic['Research Institution Phone'] = filterHTMLstr(smbAndRIInfo[5])
    dic['Principal Investigator_Name'] = filterHTMLstr(getInvestigatorInfo(bs)[0])
    dic['Principal Investigator_E-mail'] = filterHTMLstr(getInvestigatorInfo(bs)[1])
    dic['Is Principal Investigator US Citizen or Permanent Resident'] = filterHTMLstr(getInvestigatorInfo(bs)[2])
    return dic

In [62]:
Directory_path = "../Datasets/03/sttr/phase1"
files_position = ReadFiles(Directory_path)
totaldata = MultipleFileProcess(files_position)
to_CSV(totaldata)

Total number of HTML files:  45
../Datasets/03/sttr/phase1/STTR-03-1-T2.01-9798.html


NameError: name 'getSMBAndRIInfo' is not defined

# Test

In [2]:
path = "../Datasets/03/sttr/phase1/STTR-03-1-T1.01-9834.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式

In [5]:
bs.find_all("p")[4].get_text()

'\nPOTENTIAL NON-NASA COMMERCIAL APPLICATIONS (LIMIT 150 WORDS)\nThe proposed  Autonomous Coordinated Control  And Management system and related technologies are critical for realizing the vision of using a fleet of low-cost UAVs or spacecraft to replace expensive vehicles in many Earth observing or space exploration missions. The technologies developed under this project are expected to predict and prevent mechanical and software malfunctions, and reconfigure the control when they happen. Our technologies will also allow the vehicles to make intelligent decisions at higher-level autonomously, which will lead to higher mission success rate.\n'

In [55]:
### Technical Abstract and Potential Commercial Application and NASA Application
def getTechAbstract(bs):
    technicalAbstract = bs.find_all("p")[2].get_text()
    return technicalAbstract

def getNonNASAApplication(bs):
    nonNASAApplication = bs.find_all("p")[4].get_text()
    return nonNASAApplication

def getNASAApplication(bs):
    nonNASAApplication = bs.find_all("p")[3].get_text()
    return nonNASAApplication

In [60]:
bs.find_all("tr")[2].find_all("td")[1].get_text().strip()

'Coordinated Control of Multi-Agent Systems in Rapidly Varying Environments'

In [61]:
### Proposal Number
def getProposalNumber(bs):
    proposalNumber = bs.find_all("tr")[0].find_all("td")[1].get_text().strip()
    return proposalNumber

### Research Topic and project title
def getResearchSubTopicTitle(bs):
    researchTopic = bs.find_all("tr")[1].find_all("td")[1].get_text().strip()
    return researchTopic

def getProjectTitle(bs):
    projectTitle = bs.find_all("tr")[2].find_all("td")[1].get_text().strip()
    return projectTitle

In [63]:
### Small Business and Investigator Info
def getSMBAndRIInfo(bs):
    table = bs.find_all("table")[1]
    regNames = re.compile(r'NAME: *(.*?) *\n', re.S)
    names = re.findall(regNames, table.get_text())
    smbName = names[0]
    riName = names[1]

    regAddresses = re.compile(r'ADDRESS: *(.*?) *\n', re.S)
    addresses = re.findall(regAddresses, table.get_text())
    smbAddress = addresses[0]
    riAddress = addresses[1]

    regCities = re.compile(r'CITY: *(.*?) *\n', re.S)
    cities = re.findall(regCities, table.get_text())
    smbCity = cities[0]
    riCity = cities[1]

    regStates = re.compile(r'STATE/ZIP: *(.*?) \xa0', re.S)
    states = re.findall(regStates, table.get_text())
    smbState = states[0]
    riState = states[1]

    regZips = re.compile(r'STATE/ZIP:.*? \xa0 *(.*?)\n', re.S)
    zips = re.findall(regZips, table.get_text())
    smbZip = zips[0]
    riZip = zips[1]

    regPhones = re.compile(r'PHONE: *(.*?) *\n')
    phones = re.findall(regPhones, table.get_text())
    smbPhone = phones[0]
    riPhone = phones[1]
    
    smbAddress = smbAddress + ", " + smbCity + ", " + smbState + ", " + smbZip
    riAddress = riAddress + ", " + riCity + ", " + riState + ", " + riZip
    return [smbName, smbAddress, smbPhone, riName, riAddress, riPhone]

In [41]:
bs.find_all("p")[1].get_text()

'\nPRINCIPAL INVESTIGATOR/PROJECT MANAGER\n(Name,Email)\nRaman\xa0 \xa0Mehra \nrkm@ssci.com \nU.S. Citizen or Legal Resident:  Yes \n'

In [51]:
def getInvestigatorInfo(bs):
    info = bs.find_all("p")[1].get_text()
    regName = re.compile(r'Name,Email.\n *(.*?) *\n', re.S)
    name = re.findall(regName, info)
    regEmail = re.compile(r'Name,Email.\n *.*? *\n(.*?) *\nU.S. Citizen or Legal Resident', re.S)
    email = re.findall(regEmail, info)
    regIsUSCitizenOrPermanentResident = re.compile(r'U.S. Citizen or Legal Resident: *(.*?) *\n')
    isUSCitizenOrPermanentResident = re.findall(regIsUSCitizenOrPermanentResident, info)
    return [name, email, isUSCitizenOrPermanentResident]